## 作業目標: 了解斷詞演算法的背後計算

### 根據課程講述的內容, 請計算出下列剩餘所有情況的
若有一個人連續觀察到三天水草都是乾燥的(Dry), 則這三天的天氣機率為何？(可參考講義第13頁)
(Hint: 共有6種可能機率)

```python
states = ('sunny', 'rainy')
observations = ('dry', 'damp', 'soggy')
start_probability = {'sunny': 0.4, 'rainy': 0.6}
transition_probability = {'sunny':{'sunny':0.6, 'rainy':0.4},
                          'rainy': {'sunny':0.3, 'rainy':0.7}}
emission_probatility = {'sunny': {'dry':0.6, 'damp':0.3, 'soggy':0.1},
                        'rainy': {'dry':0.1, 'damp':0.4, 'soggy':0.5}}
```

```
觀察狀態 = 'dry', 'dry', 'dry'
Sunny, Sunny, Sunny: 0.4*(0.6)*0.6*(0.6)*0.6*(0.6) = 0.031104
Rainy, Sunny, Sunny: 0.6*(0.1)*0.3*(0.6)*0.6*(0.6) = 0.003888
SSS
RSS


RRS: 0.6*(0.1)*0.7*0.1*0.3*0.6     = 0.000755
RSR: 0.6*(0.1)*0.3*0.6*0.4*0.1     = 0.000432
RRR: 0.6*(0.1)*0.7*0.1*0.7*0.1     = 0.000294

SRR: 0.4*(0.6)*0.4*0.1*0.7*0.1     = 0.000672
SRS: 0.4*(0.6)*0.4*0.1*0.3*0.6     = 0.001728
SSR: 0.4*0.6*0.6*0.6*0.4*0.1       = 0.003456


start_p * trans_p * trans_p * emit_p * trans_p * emit_p 

最大機率為: Sunny, Sunny, Sunny
```

### 根據上述條件, 寫出Viterbi應用程式

In [26]:
observations = ('dry', 'dry', 'dry') #實際上觀察到的狀態為dry, dry, dry
states = ('sunny', 'rainy')
start_probability = {'sunny': 0.4, 'rainy': 0.6}
transition_probability = {'sunny':{'sunny':0.6, 'rainy':0.4},
                          'rainy': {'sunny':0.3, 'rainy':0.7}}
emission_probatility = {'sunny': {'dry':0.6, 'damp':0.3, 'soggy':0.1},
                        'rainy': {'dry':0.1, 'damp':0.4, 'soggy':0.5}}


# SSR: 0.4*0.6*0.6*0.6*0.4*0.1       = 0.003456
# start_p * trans_p * trans_p * emit_p * trans_p * emit_p 

In [44]:
def viterbi(obs, states, start_p, trans_p, emit_p):
    V = [{}]
    path = {}

    # Initialize base cases (t == 0)
    for y in states:
        V[0][y] = start_p[y] * emit_p[y][obs[0]]
#       V = [{'sunny': 0.xx, 'rainy': 0.xx}, {..}]
        path[y] = [y]
#       path = {'sunny': 'sunny'}


    # Run Viterbi for t > 0    
    for t in range(1,len(obs)):
        newpath = {}
        V += [{}]
        
        for current in states:
            (prob, state) = max([(V[t-1][last_state] * trans_p[last_state][current] * emit_p[current][obs[t]], last_state) for last_state in states])
            V[t][current] = prob
            newpath[current] = path[state] + [current]
            
        # Don't need to remember the old paths
        path = newpath
#       path = {'sunny': [state0, state_1, state_2], 'rainy': [_, _, _]}
#       print(path)
    
    (prob, state) = max([(V[len(obs) - 1][final_state], final_state) for final_state in states])
    return (prob, path[state])

In [45]:
result = viterbi(observations,
                 states,
                 start_probability,
                 transition_probability,
                 emission_probatility)

In [46]:
result

(0.031103999999999993, ['sunny', 'sunny', 'sunny'])